Manage Spotify Library
======================

In [1]:
import os
import spotify

sp = spotify.login(scope='playlist-read-private playlist-modify-private user-library-read user-library-modify')
user = sp.current_user()
user['display_name']   # check if login successful


                You need to set your Spotify API credentials.
                You can do this by setting environment variables like so:

                export SPOTIPY_CLIENT_ID='your-spotify-client-id'
                export SPOTIPY_CLIENT_SECRET='your-spotify-client-secret'
                export SPOTIPY_REDIRECT_URI='your-app-redirect-url'

                Get your credentials at
                    https://developer.spotify.com/my-applications
            


FileNotFoundError: [Errno 2] No such file or directory: '.cache-mbirgi'

In [2]:
# get user library from Spotify
albums = spotify.get_saved_albums(sp)
# hack (mongodb import limit)
albums = albums[:100]
tracks = spotify.get_saved_tracks(sp)
playlists = spotify.get_playlists(sp)
# library = {
#     "albums": albums,
#     "tracks": tracks,
#     "playlists": playlists
# }

In [3]:
# write library to files
import json
with open('library/albums.json', 'w') as fp:
    json.dump(albums, fp)
with open('library/tracks.json', 'w') as fp:
    json.dump(tracks, fp)
with open('library/playlists.json', 'w') as fp:
    json.dump(playlists, fp)

In [7]:
import pandas as pd

# df = pd.DataFrame(library)
# df

ValueError: arrays must all be same length

In [ ]:
album_results = sp.current_user_saved_albums()
album_results.keys()

In [ ]:
album_results['total']

In [ ]:
albums = album_results['items']
while album_results['next']:
    album_results = sp.next(album_results)
    albums.extend(album_results['items'])
len(albums)

In [ ]:
user_album_tracks = 0
for album in albums:
    user_album_tracks += album['album']['total_tracks']

In [ ]:
user_album_tracks

In [ ]:
track_results = sp.current_user_saved_tracks(limit=50)  # max allowed is 50
track_results.keys()

In [ ]:
track_results['total']

#### delete tracks from saved tracks that are also saved as a whole album:

In [ ]:
# get all tracks:
tracks = track_results['items']
while track_results['next']:
    track_results = sp.next(track_results)
    tracks.extend(track_results['items'])
print(f"fetched {len(tracks)} tracks")

In [ ]:
albums[0]['album'].keys()

In [ ]:
albums[0]['album']['tracks']['items'][0].keys()

In [ ]:
album = albums[0]['album']
album['artists'][0]['name']

In [ ]:
manual_check = []
for album in albums:
    album_tracks = album['album']['tracks']['items']
    track_ids = [track['id'] for track in album_tracks]
    if not album['album']['total_tracks'] == len(track_ids):
        results = sp.album_tracks(album['album']['id'])
        tracks = results['items']
        while results['next']:
            results = sp.next(results)
            tracks.extend(results['items'])
        track_ids = [track['id'] for track in tracks]
    try:
        assert album['album']['total_tracks'] == len(track_ids)
    except:
        print(f"Total tracks error: {album['album']['name']}' by '{album['album']['artists'][0]['name']}'\
                with id {album['album']['id']}")
        continue
#     print(f"track ids: {track_ids}")
    if len(track_ids) <= 50:
        if all(sp.current_user_saved_tracks_contains(track_ids)):
            sp.current_user_saved_tracks_delete(track_ids)
            print(f"deleted all {len(track_ids)} tracks from album '{album['album']['name']}' by '{album['album']['artists'][0]['name']}'")
    else:
        manual_check.append(album)

In [ ]:
[(album['album']['id'], album['album']['name']) for album in manual_check]

In [ ]:
sp.current_user_saved_albums_delete(['3HZTdHBfzDGAi7fKEDghgL'])

In [ ]:
# spotify:album:5uD7yoQZensEeRQow76SeW spotify:album:6d5jkRQxNth7GmOgibfkPX